# Import Libraries

In [1]:
import google.generativeai as genai  
from dotenv import load_dotenv
import os

c:\Users\ghost\miniconda3\envs\de_en\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)  

In [4]:

model = genai.GenerativeModel("gemini-2.0-flash")


In [5]:
response = model.generate_content("As a German teacher, explain the grammar behind the sentence 'Haben Sie Ihre Online-Bewerbung ausgefüllt?' with a detailed breakdown. Include the roles of each word in the sentence, the word order rules in German questions, and any key grammar points related to this structure.")

In [6]:
print(response.text)

Okay, let's break down the German sentence "Haben Sie Ihre Online-Bewerbung ausgefüllt?" ("Have you filled out your online application?") in a way that explains the grammar thoroughly.

**Translation:**

*   "Haben Sie Ihre Online-Bewerbung ausgefüllt?" = "Have you filled out your online application?"

**Word-by-Word Breakdown & Grammatical Roles:**

*   **Haben:**  Auxiliary Verb (Hilfsverb).
    *   This is the conjugated form of the verb "haben" (to have) in the present tense, specifically conjugated for "Sie" (formal "you").
    *   *Role:*  In this sentence, "haben" functions as the *auxiliary verb* needed to form the perfect tense (Perfekt).  The Perfekt is a common past tense form in spoken German.

*   **Sie:** Personal Pronoun (Personalpronomen).
    *   This is the formal "you" in German.  Remember, German has "du" (informal "you") and "Sie" (formal "you").
    *   *Role:*  Subject of the sentence.  It's the person *doing* the action.

*   **Ihre:** Possessive Pronoun (Posses